In [7]:
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from urllib.request import urlopen
from time import sleep

In [8]:
def process_browser_log_entry(entry):
    response = json.loads(entry['message'])['message']
    return response

In [10]:
days_pending2now_df = pd.DataFrame()

# Set up Chrome to capture performance logs
options = webdriver.ChromeOptions()
options.set_capability('goog:loggingPrefs', {'performance': 'ALL'})

driver = webdriver.Chrome(options=options)

driver.get("https://trac.syr.edu/phptools/immigration/asylumbl/")

wait = WebDriverWait(driver, 10)  # wait for up to 10 seconds
days_pending2now_button = wait.until(EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div[2]/div/div/div/div/div/div[2]/div/div[2]/div[1]/div[2]/div/div[2]/div[2]/input')))
days_pending2now_button.click()

wait = WebDriverWait(driver, 10)  # wait for up to 10 seconds
state_table = wait.until(EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div[2]/div/div/div/div/div/div[3]/div/div[1]/div[2]/div/div/div/table')))
state_table_elements = state_table.find_element(By.TAG_NAME, 'tbody').find_elements(By.TAG_NAME, 'tr')

# We don't want to filter out "All" category for days waited
#filtered_state_table_elements = [elem for elem in state_table_elements if "All-" not in elem.text]
filtered_state_table_elements = state_table_elements

for state_elem in filtered_state_table_elements:
    state_elem.click()

    sleep(2)
    wait = WebDriverWait(driver, 10)  # wait for up to 10 seconds
    court_table = wait.until(EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div[2]/div/div/div/div/div/div[3]/div/div[2]/div[2]/div/div/div/table')))
    court_table_elements = court_table.find_element(By.TAG_NAME, 'tbody').find_elements(By.TAG_NAME, 'tr')
    
    # We don't want to filter out "All" category for days waited
    #filtered_court_table_elements = [elem for elem in court_table_elements if "All-" not in elem.text]
    filtered_court_table_elements = court_table_elements

    for court_elem in filtered_court_table_elements:
        if ("All" in state_elem.text and "All" not in court_elem.text):
            continue
        
        court_elem.click()

        sleep(2)
        wait = WebDriverWait(driver, 10)  # wait for up to 10 seconds
        table = wait.until(EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div[2]/div/div/div/div/div/div[3]/div/div[3]/div[2]/div/div/div/table')))
        tbody_elements = table.find_element(By.TAG_NAME, 'tbody').find_elements(By.TAG_NAME, 'tr')
        
        # We don't want to filter out "All" category for days waited
        #filtered_elements = [elem for elem in tbody_elements if "All-" not in elem.text]
        filtered_elements = tbody_elements

        # Iterate through the list of elements
        for elem in filtered_elements:
            elem.click()
            
            sleep(2)
            # Get the network logs
            browser_log = driver.get_log('performance') 
            events = [process_browser_log_entry(entry) for entry in browser_log]
            network_responses = [event for event in events if 'Network.responseReceived' in event['method']]
            
            # Assuming the most recent response is the last in the list
            if network_responses:
                url = network_responses[-1]['params']['response']['url']
            
                try: 
                    response = urlopen(url)
                except:
                    continue

                data_json = json.loads(response.read())

                #print(data_json)

                if data_json['timeline']:
                    print(data_json)
                    df = pd.DataFrame(data_json['timeline'])

                    city, nationality = data_json['title'].rsplit(' - ', 1)

                    df['city'] = city
                    df['nationality'] = nationality

                    days_pending2now_df = pd.concat([days_pending2now_df, df])

{'title': 'All - All - All', 'x_axis_label': 'Fiscal Year', 'y_axis_label': {'number': 'Average Days Waited So Far', 'percent': 'Percent'}, 'timeline': [{'pendfy': '2012', 'number': '665.0', 'percent': '100.00000'}, {'pendfy': '2013', 'number': '698.0', 'percent': '100.00000'}, {'pendfy': '2014', 'number': '741.0', 'percent': '100.00000'}, {'pendfy': '2015', 'number': '766.0', 'percent': '100.00000'}, {'pendfy': '2016', 'number': '779.0', 'percent': '100.00000'}, {'pendfy': '2017', 'number': '727.0', 'percent': '100.00000'}, {'pendfy': '2018', 'number': '766.0', 'percent': '100.00000'}, {'pendfy': '2019', 'number': '807.0', 'percent': '100.00000'}, {'pendfy': '2020', 'number': '939.0', 'percent': '100.00000'}, {'pendfy': '2021', 'number': '1156.0', 'percent': '100.00000'}, {'pendfy': '2022', 'number': '1114.0', 'percent': '100.00000'}, {'pendfy': '2023', 'number': '944.0', 'percent': '100.00000'}]}
{'title': 'All - All - St. Kitts, West Indies', 'x_axis_label': 'Fiscal Year', 'y_axis_l

WebDriverException: Message: disconnected: Unable to receive message from renderer
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=116.0.5845.188)
Stacktrace:
	GetHandleVerifier [0x00007FF7B00952A2+57122]
	(No symbol) [0x00007FF7B000EA92]
	(No symbol) [0x00007FF7AFEDE3AB]
	(No symbol) [0x00007FF7AFECC01A]
	(No symbol) [0x00007FF7AFECBF63]
	(No symbol) [0x00007FF7AFECA9E5]
	(No symbol) [0x00007FF7AFECB31E]
	(No symbol) [0x00007FF7AFEE0A37]
	(No symbol) [0x00007FF7AFEE1268]
	(No symbol) [0x00007FF7AFEE3D2F]
	(No symbol) [0x00007FF7AFEE3E20]
	(No symbol) [0x00007FF7AFF197A4]
	(No symbol) [0x00007FF7AFF18DF3]
	(No symbol) [0x00007FF7AFF0F25F]
	(No symbol) [0x00007FF7AFF36FDA]
	(No symbol) [0x00007FF7AFF0EB76]
	(No symbol) [0x00007FF7AFF371F0]
	(No symbol) [0x00007FF7AFF4EB82]
	(No symbol) [0x00007FF7AFF36DB3]
	(No symbol) [0x00007FF7AFF0D2B1]
	(No symbol) [0x00007FF7AFF0E494]
	GetHandleVerifier [0x00007FF7B033EF82+2849794]
	GetHandl

In [ ]:
days_pending2now_df.to_csv("data/days_pending2now_df.csv")